# Perbedaan NLTK dan Spacy

Spacy is better than NLTK in terms of performance.Here, there are some comparison

1- NLTK is a string processing library. It takes strings as input and returns strings or lists of strings as output.Whereas, spaCy uses object-oriented approach.When we parse a text, spaCy returns document object whose words and sentences are objects themselves.

2- spaCy has support for word vectors whereas NLTK does not.

3- In word tokenization and POS-tagging spaCy performs better, but in sentence tokenization, NLTK outperforms spaCy.

4- NLTK supports various languages whereas spaCy have statistical models for 7 languages (English, German, Spanish, French, Portuguese, Italian, and Dutch). It also supports named entities for multi language.

## memakai NLTK

In [1]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [2]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [3]:
teks_nltk = word_tokenize_wrapper('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

In [4]:
teks_nltk

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

## memakai spacy

lakukan installasi dan update lib spacy

In [5]:
from spacy.lang.id import Indonesian
import spacy

nlp = Indonesian()  # use directly
nlp = spacy.blank('id')  # blank instance'
Teks = nlp('Yahya beserta teman-teman TK suka melihat lumba-lumba di Batang Dolphin Center')

In [6]:
Token_kata = [token.text for token in Teks]

In [7]:
Token_kata

['Yahya',
 'beserta',
 'teman-teman',
 'TK',
 'suka',
 'melihat',
 'lumba-lumba',
 'di',
 'Batang',
 'Dolphin',
 'Center']

# load library

In [8]:
import re
import string
import time
from copy import deepcopy

# load dataset

dataset ini menggunakan sentimen dan emosi.

kali ini kita akan menggunakan sentimen terlebih dahulu

silakan drop kolom Emosi

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('Dataset_Sentimen_Emosi.csv')

In [11]:
df.head()

,Tweet,Sentimen,Emosi
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,1
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,-1
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,1
3,Covid belum nyampe prigen mbak hmm hoax,0.0,-2
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,-2


In [12]:
df = df.drop(['Emosi'], axis=1)

In [13]:
df

,Tweet,Sentimen
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0
3,Covid belum nyampe prigen mbak hmm hoax,0.0
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0
...,...,...
899,Seluruh negara di dunia mengalami masa sulit k...,1.0
900,"Setelah covid dan skripsi disaster selesai, ma...",1.0
901,"Malam ini!! Projek ""BENDA BOLEH BINCANG"" 9 mal...",0.0
902,Pontang - panting di koyak covid 19,-1.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     904 non-null    object 
 1   Sentimen  903 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.2+ KB


# Preprocessing dataset
kita akan pakai modul dari AMS 01-03

disamping itu kita akan memakai modul  https://github.com/cbaziotis/ekphrasis 

untuk instalasi, silakan buka tanda pagar dan jalankan

In [15]:
#!pip install ekphrasis

In [16]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


Word statistics files not found!
Downloading... 

/home/aldiazmi/Jupyter-Notebook/Minggu2/textProcessing/virtualenv/lib/python3.10/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


KeyboardInterrupt: 

In [ ]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [ ]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('rt'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [ ]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## lakukan pembersihan dengan memanggil fungsi yang didefinisikan diatas

In [ ]:
i = 0
final_string = []
s = ""
for text in df['Tweet'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = lower(text)
    proc = change_stripe(text)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    #proc = remove_rt(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [ ]:
df["step01"] = final_string

In [ ]:
df.head(10)

## hapus data kosong

kadang ada data yang kosong. dan ini tidak bisa kita apa-apakan, maka dari itu kita hapus saja

In [ ]:
df.info()

In [ ]:
df_hapus = df[~df['step01'].str.contains(" ")]

In [ ]:
df_hapus.info()

In [ ]:
df_hapus.head(10)

In [ ]:
df_new = df[~df.isin(df_hapus)].dropna()

In [ ]:
df_new.info()

In [ ]:
df_new

## normalisasi kata slang menjadi kata baku

In [ ]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [ ]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [ ]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [ ]:
df_new.head(10)

In [ ]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [ ]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

In [ ]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [ ]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [ ]:
df_new["step02"] = final_string_tokens

In [ ]:
df_new.head(10)

# simpan

In [ ]:
df.to_csv('clean_dataset2.csv',sep=";")